# TM Forum Guru - Resoning Engine based Agent

## Dependencies

In [ ]:
%pip install google-cloud-discoveryengine --upgrade --user
%pip install --upgrade google-auth
!pip install --upgrade --quiet \
    google-cloud-aiplatform==1.51.0 \
    langchain==0.1.20 \
    langchain-google-vertexai==1.0.3 \
    cloudpickle==3.0.0 \
    pydantic==2.7.1 \
    langchain_google_community \
    google-cloud-discoveryengine \
    google-api-python-client \
    requests \
    ratelimit \
    python-dotenv

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 2.29.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [ ]:
## All the Required imports
from google.api_core.client_options import ClientOptions
from google.cloud import discoveryengine_v1 as discoveryengine
from dotenv import load_dotenv, find_dotenv
from vertexai.preview import reasoning_engines
from googleapiclient import discovery
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.memory import ChatMessageHistory
from operator import itemgetter
from typing import List
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.documents import Document
from langchain_core.messages import BaseMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import (
    RunnableLambda,
    ConfigurableFieldSpec,
    RunnablePassthrough,
)
from langchain_google_vertexai import HarmBlockThreshold, HarmCategory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core import prompts
from langchain_core import agents
from langchain.agents.format_scratchpad import (
    format_to_openai_function_messages
)

import google.auth
import json
import time
import os
import vertexai
import requests

## Environment variables

In [ ]:
## Environment Variables needs to be set explicitly based on the env(dev/SIT)
# !export PROJECT_ID="enterprise-search-gen-ai"
# !export SEARCH_APP_LOCATION="global"
# !export STAGING_BUCKET="gs://agent-test-srini"
# !export DATA_STORE_ID_PDF="tmf-metadata-layout-p_1715009821486"
# !export DATA_STORE_ID_WEB="tmf-public_1692445422672"
# !export AGENT_LOCATION="us-central1"

In [ ]:
# Load environment variables
_=load_dotenv(find_dotenv())
credentials, _ = google.auth.default()
request = google.auth.transport.requests.Request()
credentials.refresh(request)
AUTH_TOKEN = credentials.token
PROJECT_ID = os.getenv('PROJECT_ID') if os.getenv('PROJECT_ID') else 'enterprise-search-gen-ai'
SEARCH_APP_LOCATION = os.getenv('SEARCH_APP_LOCATION') if os.getenv('SEARCH_APP_LOCATION') else 'global'
AGENT_LOCATION = os.getenv('AGENT_LOCATION') if os.getenv('AGENT_LOCATION') else 'us-central1'
STAGING_BUCKET = os.getenv('STAGING_BUCKET') if os.getenv('STAGING_BUCKET') else 'gs://agent-test-srini'
DATA_STORE_ID_PDF = os.getenv('DATA_STORE_ID_PDF') if os.getenv('DATA_STORE_ID_PDF') else "tmf-metadata-layout-p_1715009821486"
DATA_STORE_ID_WEB = os.getenv('DATA_STORE_ID_WEB') if os.getenv('DATA_STORE_ID_WEB') else "tmf-public_1692445422672"

In [ ]:
print(PROJECT_ID)

enterprise-search-gen-ai


In [ ]:
safety_settings = {
    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
}

# model ="gemini-1.5-pro-latest"
AGENT_LLM_MODEL = "gemini-1.5-flash-001" # "gemini-1.5-pro" # "gemini-1.5-flash-preview-0514" # "gemini-1.5-pro-preview-0409"

# model configuration
AGENT_LLM_MODEL_KWARGS = {
    # temperature (float): The sampling temperature controls the degree of
    # randomness in token selection.
    "temperature": 0.28,
    # max_output_tokens (int): The token limit determines the maximum amount of
    # text output from one prompt.
    "max_output_tokens": 1000,
    # top_p (float): Tokens are selected from most probable to least until
    # the sum of their probabilities equals the top-p value.
    "top_p": 0.95,
    # top_k (int): The next token is selected from among the top-k most
    # probable tokens.
    "top_k": 40,
    "safety_settings": safety_settings,
    # safety_settings (Dict[HarmCategory, HarmBlockThreshold]): The safety
    # settings to use for generating content.\
}

## Vertex AI tools

### GCP search datastore tool

In [ ]:
def search_data_store(
    query_input: str,
    auth_token: str,
    project_id: str,
    location_id: str,
    datastore_id: str,
    page_size: int,
    llm_model_version: str,
    ):
#)-> str:


    """Looks up for things in pdf document stored  related to tmf forum"""

    import requests
    import json

    # @markdown ### API Parameters
    asynchronous_mode = False # @param ["False", "True"] {type:"raw"}

    # @markdown SAFETY_SPEC
    safe_search = False # @param ["False", "True"] {type:"raw"}

    # @markdown RELATED_QUESTION_SPEC
    related_questions = True # @param ["False", "True"] {type:"raw"}

    # @markdown QUERY_UNDERSTANDING_SPEC
    disable_query_rephraser = True # @param ["False", "True"] {type:"raw"}
    max_rephrase_steps = 1 # @param {type:"slider", min:0, max:5, step:1}

    # @markdown SEARCH_SPEC
    max_return_results = 3 # @param {type:"slider", min:0, max:10, step:1}
    search_filter = '' # @param {type: 'string'}

    # @markdown ANSWER_GENERATION_SPEC
    include_citations = True # @param ["False", "True"] {type:"raw"}
    ignore_adversarial_query = True # @param ["False", "True"] {type:"raw"}
    ignore_non_answer_seeking_query = True # @param ["False", "True"] {type:"raw"}
    answer_gen_preamble = '' # @param {type:"string"}
    answer_gen_language_code = '' # @param ["en", "es", "jp"] {allow-input: true}
    # A couple options for the model: https://cloud.google.com/generative-ai-app-builder/docs/answer-generation-models
    answer_gen_version = 'gemini-1.0-pro-002/answer_gen/v1' # @param ["preview", "stable", "text-bison@001/answer_gen/v1", "text-bison@002/answer_gen/v1", "gemini-1.0-pro-001/answer_gen/v1", "gemini-1.0-pro-002/answer_gen/v1", "gemini-1.5-pro-001/answer_gen/v1"] {allow-input: true}

    resp = requests.post(
      f'https://discoveryengine.googleapis.com/v1alpha/projects/{project_id}/locations/{location_id}/collections/default_collection/dataStores/{datastore_id}/servingConfigs/default_search:answer',
      headers={
        'Content-Type': 'application/json',
        'Authorization': 'Bearer ' + auth_token,
      },
      json={
    "query": { "text": query_input},

        # Return Related / Followup questions if set to True.
        "relatedQuestionsSpec": {
            "enable": related_questions,
        },


        "queryUnderstandingSpec": {
          # Query rephraser that transfer the raw query to
          # 1. shorter search query
          # 2. multiple queries
          # 3. multi-steps queries (that could run search multi-times based on search results)
          "queryRephraserSpec": {
              "disable": disable_query_rephraser, # disable the rephraser
              "maxRephraseSteps": max_rephrase_steps,
          },
        },


        "searchSpec": {
            "searchParams": {
                # Max search results to return by search.
                "maxReturnResults": max_return_results,
                # same as search API
                "filter": search_filter,
            },
        },


        "answerGenerationSpec": {
          # enable the citations in the output
          "includeCitations": include_citations,

          # force the answer to be generated in a target language.
          "answerLanguageCode": answer_gen_language_code,

          "modelSpec": {
              # A couple options for the model: https://cloud.google.com/generative-ai-app-builder/docs/answer-generation-models
              "modelVersion": answer_gen_version,
          },

          # Customer preamble that could be put in the LLM input.
          # Example Preamble to test
          # Given the conversation between a user and a helpful assistant and some search results, create a final answer for the assistant.
          # The answer should use all relevant information from the search results, not introduce any additional information,
          # and use exactly the same words as the search results when possible. The assistant’s answer should be no more than 20 sentences.
          # The user is an expert who has an in-depth understanding of the subject matter.
          # The assistant should answer in a technical manner that uses specialized knowledge and terminology when it helps answer the query.
          "promptSpec":{ "preamble": answer_gen_preamble },

          # Do not generate answer for adversarial queries
          "ignoreAdversarialQuery": ignore_adversarial_query,

          # Do not generate answer for non-answer seeking queries
          "ignoreNonAnswerSeekingQuery": ignore_non_answer_seeking_query,
        },

        "safetySpec": {
            "enable": safe_search,
        },

        "asynchronousMode": asynchronous_mode,
      },
    )
    return resp.json()

In [ ]:
def parse_search_tool_response(response):
  # check if tool has a answer in the response
  if "answer" in response:
    answer_object = response["answer"]
    print(answer_object.keys())
    # successful retrieval
    if "state" in answer_object and answer_object["state"] == "SUCCEEDED":
      answerText = answer_object["answerText"]
      citations = answer_object["citations"]
      references = answer_object["references"]
      related_questions = answer_object["relatedQuestions"]
      print(answerText)

### GCP Search image tool

In [ ]:
def gcp_webImageSearch(
    query_input: str,
    auth_token: str,
    project_id:str,
    location_id: str,
    app_id: str,
    summary_result_size:int
):
    """Looks up for images related to the query input in the TM Forum web pages"""

    import requests
    import json

    END_POINT_URL = f'https://discoveryengine.googleapis.com/v1/projects/{project_id}/locations/{location_id}/collections/default_collection/engines/{app_id}/servingConfigs/default_config:search'
    headers = {'Content-type': 'application/json',
               'Authorization':f'Bearer {auth_token}',
               'X-Goog-User-Project': f'{project_id}'}
    data = {
      "servingConfig": f'projects/{project_id}/locations/global/collections/default_collection/engines/tmf-public_1692445422672/servingConfigs/default_search',
      "query": f'{query_input}',
      "pageSize": summary_result_size,
      "params": {"search_type": 1}
    }

    r = requests.post(END_POINT_URL, data=json.dumps(data), headers=headers)
    return r.json()

In [ ]:
import pprint
credentials, _ = google.auth.default()
request = google.auth.transport.requests.Request()
credentials.refresh(request)

res = gcp_webImageSearch(
  query_input="what is ODF?",
   auth_token=credentials.token,
   project_id=PROJECT_ID,
   location_id=SEARCH_APP_LOCATION,
   app_id="tmf-public_1692445422672",
   summary_result_size= 3,
 )

# pprint.pprint(res['results'][0]['document']['derivedStructData']['image']['contextLink'])
# pprint.pprint(res)

In [ ]:
def search_web_and_pdf(
    query_str: str):

    """This is the function to call to answer all the user questions EXCEPT code generation requests."""

    import os
    import requests
    import google.auth
    from dotenv import load_dotenv, find_dotenv
    from google.oauth2 import service_account
    from vertexai.generative_models import (
      GenerativeModel,
    )

    _=load_dotenv(find_dotenv())


    credentials, _ = google.auth.default()
    request = google.auth.transport.requests.Request() # User is handling it, user token will come, if the service account token will come.
    credentials.refresh(request)
    PROJECT_ID = os.getenv('PROJECT_ID') if os.getenv('PROJECT_ID') else 'enterprise-search-gen-ai'
    LOCATION_ID = SEARCH_APP_LOCATION = os.getenv('SEARCH_APP_LOCATION') if os.getenv('SEARCH_APP_LOCATION') else 'global'
    DATA_STORE_ID_PDF = DATA_STORE_ID_PDF = os.getenv('DATA_STORE_ID_PDF') if os.getenv('DATA_STORE_ID_PDF') else "tmf-metadata-layout-p_1715009821486" # pdf
    DATA_STORE_ID_WEB = os.getenv('DATA_STORE_ID_WEB') if os.getenv('DATA_STORE_ID_WEB') else "tmf-public_1692445422672" # WEB
    SEARCH_APP_LLM_MODEL = 'gemini-1.0-pro-002/answer_gen/v1'

    print("PROJECT_ID", PROJECT_ID)

    query = query_str

    vertexai.init(project=PROJECT_ID, location='us-central1')

    pdf_response = search_data_store(
                  query_input=query_str,
                  auth_token=credentials.token,
                  project_id=PROJECT_ID,
                  location_id=LOCATION_ID,
                  datastore_id=DATA_STORE_ID_PDF,
                  page_size= 5,
                  llm_model_version=SEARCH_APP_LLM_MODEL
                )

    web_response = search_data_store(
                  query_input=query_str,
                  auth_token=credentials.token,
                  project_id=PROJECT_ID,
                  location_id=LOCATION_ID,
                  datastore_id=DATA_STORE_ID_WEB,
                  page_size= 5,
                  #summary_result_size= 5,
                  llm_model_version=SEARCH_APP_LLM_MODEL
                )

    image_response = gcp_webImageSearch(
                              query_input=query_str,
                              auth_token=credentials.token,
                              project_id=PROJECT_ID,
                              location_id=LOCATION_ID,
                              app_id=DATA_STORE_ID_WEB,
                              summary_result_size= 3,
                      )

    consolidated_responses = [pdf_response, web_response, image_response]
    return consolidated_responses

### Swagger based Code generation tool for Open API specifications

In [ ]:
def swagger_gen(
    text: str,
):
    """
    Generates servers or clients code in the language requested for the JSON spec stored in swaggerUrl and writes it to the output file
    """
    import requests
    import json
    import google.auth.transport.requests
    import google.oauth2.id_token

    # credentials, _ = google.auth.default()
    auth_request = google.auth.transport.requests.Request() # User is handling it, user token will come, if the service account token will come.
    url= "https://us-central1-enterprise-search-gen-ai.cloudfunctions.net/swagger-code-generator-1"
    id_token = google.oauth2.id_token.fetch_id_token(auth_request, url)

    headers = {
        "Authorization": f"bearer {id_token}",
        "Content-Type": "application/json"
    }

    data = {
        "text": text
    }

    response = requests.post(url, headers=headers, json=data, timeout=70)

    return response.json()

In [ ]:
# res = swagger_gen("Can you give me the code for Product catalogue management version 4?")
# res

### Chat History

In [ ]:
class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """In memory implementation of chat message history."""

    messages: List[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: List[BaseMessage]) -> None:
        """Add a list of messages to the store"""
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []

# Here we use a global variable to store the chat message history.
# This will make it easier to inspect it to see the underlying results.
store = {}

def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]


### Create/Deploy/Redeployment Reasoning Agent

In [ ]:
vertexai.init(project=PROJECT_ID, location=AGENT_LOCATION, staging_bucket=STAGING_BUCKET)

In [ ]:
# ## This prompt template needs to be changed
# prompt = {
#     "input": lambda x: x["input"],
#     "agent_scratchpad": (
#         lambda x: format_to_openai_function_messages(x["intermediate_steps"])
#     ),
# } | prompts.ChatPromptTemplate.from_messages([
#     ("system", """
#     - Greet the users and then ask how you can help them today.
#     - You are an expert TM Forum assistant, to help people with TMForum related documentation and specifications which are available in the form of PDF and web assets.
#     - Call search_web_and_pdf whenever you are asked about some information about TM Forum organisation. Please answer the user query from all the relevant information conatained in any part of the context. Please provide detailed responses. Please add the citations in the context, in the format of [#], where # is a number, to the summary at the appropriate place. Ensure that the citation numbers you add to the summary match the citation numbers in the context.
#     - Call swagger_gen whenever you are asked to generate server or client code or any code for any open api specification.
#     Example:
#     'The code for Product catalogue management version 4 is available at https://storage.cloud.google.com/tfm-ai-assistant-codegen-dev/generatedCodeFile/TMF620_Product_Catalog_Management_API_v4_1716933037458.zip.'
#     - If necessary, seek clarifying details, Please do not make up any information.
#     - Please restrict yourself to the information related to TM Forum.
#     """),
#     ("user", "{input}"),
#     ("placeholder", "{agent_scratchpad}"),
# ])

In [ ]:
DISPLAY_NAME = "TMFMultiChatAppJune04v1"
remote_app = reasoning_engines.ReasoningEngine.create(
    reasoning_engines.LangchainAgent(
        # prompt=prompt,
        model=AGENT_LLM_MODEL,
        tools=[search_web_and_pdf,swagger_gen],
        model_kwargs=AGENT_LLM_MODEL_KWARGS,
        agent_executor_kwargs={"return_intermediate_steps": True},
    ),
    requirements=[
        "google-cloud-aiplatform==1.51.0",
        "langchain==0.1.20",
        "langchain-google-vertexai==1.0.3",
        "cloudpickle==3.0.0",
        "pydantic==2.7.1",
        "requests",
        # "google-cloud-discoveryengine",
        "google-auth",
        "python-dotenv"
    ],
    display_name=DISPLAY_NAME,
)
remote_app

INFO:vertexai.reasoning_engines._reasoning_engines:Using bucket agent-test-srini
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://agent-test-srini/reasoning_engine/reasoning_engine.pkl
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://agent-test-srini/reasoning_engine/requirements.txt
INFO:vertexai.reasoning_engines._reasoning_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://agent-test-srini/reasoning_engine/dependencies.tar.gz
INFO:vertexai.reasoning_engines._reasoning_engines:Creating ReasoningEngine
INFO:vertexai.reasoning_engines._reasoning_engines:Create ReasoningEngine backing LRO: projects/982845833565/locations/us-central1/reasoningEngines/8863400726013935616/operations/2788403578719436800
INFO:vertexai.reasoning_engines._reasoning_engines:ReasoningEngine created. Resource name: projects/982845833565/locations/us-central1/reasoningEngines/8863400726013935616
INFO:vertexai.r

resource name: projects/982845833565/locations/us-central1/reasoningEngines/8863400726013935616

In [ ]:
import pprint
try:
  res = remote_app.query(input="what is ODA?")
  pprint.pprint(res)
except Exception as e:
   pprint.pprint(e)

Streaming output truncated to the last 5000 lines.
                                                                               'component '
                                                                               'directory '
                                                                               'Running '
                                                                               'on '
                                                                               'ODA '
                                                                               'Ready '
                                                                               'for '
                                                                               'ODA '
                                                                               'Governance '
                                                                               'Business '
                                                            

### Testing Reasoning Engine

In [1]:
import google.auth
import json
import requests

def call_tmf_aiva_agent(query_str, token):
  """Makes a POST request to the specified Reasoning Engine endpoint."""
  # Set the headers and data
  headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json; charset=utf-8",
  }

  # Create the JSON payload from the query_str
  json_input = {"input": {"input": f"{query_str}"}}

  # Construct the URL
  location = "us-central1"
  project_id = "982845833565"
  location_id = "us-central1"
  reasoning_engine_id = "6866054291275120640"
  url = f"https://{location_id}-aiplatform.googleapis.com/v1beta1/projects/{project_id}/locations/{location_id}/reasoningEngines/{reasoning_engine_id}:query"

  # Send the POST request
  r = requests.post(url, headers=headers, data=json.dumps(json_input))
  return r.json()

In [2]:
credentials, _ = google.auth.default()
request = google.auth.transport.requests.Request()
credentials.refresh(request)
token = credentials.token

In [3]:
queries = [
    "Describe ODF",
    "Can you give me the code for Product catalogue management version 4? use swagger gen tool",
    "Are TM Forum and MEF APIs the same?"
]

In [4]:
res = call_tmf_aiva_agent(queries[0], token)
print(res)

{'output': {'input': 'Describe ODF', 'intermediate_steps': [[{'id': ['langchain', 'schema', 'agent', 'ToolAgentAction'], 'type': 'constructor', 'kwargs': {'tool_call_id': '18ee80c5-0fd1-4f89-b78e-3d2124f864d8', 'tool_input': {'query_str': 'Describe ODF'}, 'type': 'AgentActionMessageLog', 'message_log': [{'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessageChunk'], 'kwargs': {'type': 'AIMessageChunk', 'additional_kwargs': {'function_call': {'name': 'search_web_and_pdf', 'arguments': '{"query_str": "Describe ODF"}'}}, 'tool_calls': [{'name': 'search_web_and_pdf', 'id': '18ee80c5-0fd1-4f89-b78e-3d2124f864d8', 'args': {'query_str': 'Describe ODF'}}], 'content': '', 'tool_call_chunks': [{'id': '18ee80c5-0fd1-4f89-b78e-3d2124f864d8', 'name': 'search_web_and_pdf', 'args': '{"query_str": "Describe ODF"}', 'index': None}], 'invalid_tool_calls': [], 'response_metadata': {'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'blocked': False, 'probability_label': 'NE

### Un-deployment of Reasoning Engine

In [5]:
import time
import vertexai
from vertexai.preview import reasoning_engines

In [6]:
def delete_reasoning_engine(project_id, location, engine, rate_limit_time=30):
  try:
    vertexai.init(project=project_id, location=location)
    reasoning_engine = reasoning_engines.ReasoningEngine(engine)
    reasoning_engine.delete()
    time.sleep(rate_limit_time)
    return True
  except Exception as exc:
    return False

In [7]:
# engine to delete
PROJECT_ID = "982845833565"
LOCATION = "us-central1"
REASONING_ENGINE_ID = "1304953176402690048"

In [8]:
status = delete_reasoning_engine(PROJECT_ID, LOCATION, REASONING_ENGINE_ID)
print(status)

False
